In [19]:
import sys,os

#sys.path.append('/home/arash/VRdataCleaning/DeepSurv/')

import importlib
import deepsurv
from statsmodels.stats.outliers_influence import variance_inflation_factor    
import argparse
import uuid
import pickle
import json
import time
import numpy as np
import pandas as pd

import lasagne
import optunity

import logging
from logging import handlers
from sklearn.model_selection import train_test_split

importlib.reload(deepsurv)

from deepsurv import deep_surv, utils

from deepsurv.deepsurv_logger import DeepSurvLogger, TensorboardLogger
from eli5.permutation_importance import get_score_importances
import shap  # package used to calculate Shap values
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

WARNING (theano.configdefaults): install mkl with `conda install mkl-service`: No module named 'mkl'
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


## Load data: 
#### xtrain: 80% of all data, devided into xtrainsub & xvalsub (80 and 20%) 
#### xtest: 20% of all data

In [20]:
with open('/home/arash/ProjectVR/cleaneddata/deepwaitdata/xtrain', 'rb') as f:
    xtrain=pickle.load(f)
    
data=data.loc[-data['Wait Time (s)'].isin(['Err1','Err2','Err3','Err4','Err5'])]
data=data.loc[data['Age_9-12'].isin([0,1])]


data=data.loc[:,['Wait Time (s)','Speed Limit', 'Lane Width', 'Minimum Gap', 'Mean Arrival Rate', 'AV', 
                   'Full Braking Before Impact_-1.0', 'Full Braking Before Impact_1', 
                   'Full Braking Before Impact_2', 'Full Braking Before Impact_3', 'Snowy',
                   'One way', 'two way', 'Two way with median', 'Night', 'numcars', 
                    'Age_9-12', 'Age_15-18', 'Age_12-15', 'Age_18 - 24', 'Age_25 - 29', 'Age_30 - 39', 'Age_50 - 59', 'Age_60+', 'Gender_Female', 'Occupation_Employed', 'Occupation_Student', 'Occupation_Unemployed', 
                   'Occupation_kid', 'Education_Bachelors degree', 'Education_College/University student', 
                   'Education_Doctorate degree', 'Education_High school diploma', 'Education_Masters degree', 
                   'Education_Professional degree', 'driving license_Yes', 'mode_Bike', 'mode_Car',
                   'mode_Public Transit', 'mode_Walking', 'workwalk_No', 'workwalk_Sometimes', 'workwalk_Yes', 
                   'shopwalk_No', 'shopwalk_Sometimes', 'shopwalk_Yes', 'Vrexp_Yes']]       #numwalk and VRexpnum removed because of some false inputs in the data should be fixed later

data=pd.get_dummies(data=data,columns=['Speed Limit','Lane Width','Minimum Gap','Mean Arrival Rate','numcars'])



data=data.apply(pd.to_numeric, errors='coerce')


def FullAV(row):
    if row['AV']==100:
        return 1
    else:
        return 0
def MixedAV(row):
    if row['AV']>0 and row['AV']<100:
        return 1
    else:
        return 0   
def HDV(row):
    if row['AV']==0:
        return 1

    else:
        return 0   
data=data[data.Occupation_kid==0]                         #remove kids


data['FullAV']=data.apply (lambda row: FullAV(row), axis=1)
data['Mixed']=data.apply (lambda row: MixedAV(row), axis=1)
#data['HDV']=data.apply (lambda row: HDV(row), axis=1)
data['over50']=data['Age_50 - 59']+data['Age_60+']
data=data.drop(columns=['Age_9-12', 'Age_15-18', 'Age_12-15','Occupation_kid','AV','Age_50 - 59','Age_60+'])
data['E']=1




data=data.drop(columns=['Full Braking Before Impact_3', 'two way', 'Occupation_Unemployed','Education_Doctorate degree','mode_Bike','workwalk_Yes','shopwalk_Yes','Speed Limit_50.0','Lane Width_3.0','Minimum Gap_2.0','Mean Arrival Rate_1100.0','numcars_4'])


train, test = train_test_split(data,test_size=0.2, random_state=1)       #20 percent of data is seperated for test set
NCtrain=train     #NC: Not changed, used later for shap, without a format change that deepsurv requires
NCtest= test
trainsub, valsub = train_test_split(train,test_size=0.2, random_state=1)   #20 percent of train data is set for validation set, for cross-validation train is used, for network training using opt_params, trainsub and valsub are used

In [21]:
with open('/home/arash/ProjectVR/cleaneddata/deepwaitdata/xtrainsub', 'rb') as f:
    xtrainsub=pickle.load(f)
    
with open('/home/arash/ProjectVR/cleaneddata/deepwaitdata/ytrainsub', 'rb') as f:
    ytrainsub=pickle.load(f)
    
with open('/home/arash/ProjectVR/cleaneddata/deepwaitdata/xvalsub', 'rb') as f:
    xvalsub=pickle.load(f)
    
with open('/home/arash/ProjectVR/cleaneddata/deepwaitdata/yvalsub', 'rb') as f:
    yvalsub=pickle.load(f) 

In [22]:
with open('/home/arash/ProjectVR/cleaneddata/deepwaitdata/nctrain', 'rb') as f:
    NCtrain=pickle.load(f)
    
with open('/home/arash/ProjectVR/cleaneddata/deepwaitdata/nctest', 'rb') as f:
    NCtest=pickle.load(f) 

In [23]:
with open('/home/arash/ProjectVR/cleaneddata/deepwaitdata/nctrain', 'rb') as f:
    NCtrain=pickle.load(f)
    
with open('/home/arash/ProjectVR/cleaneddata/deepwaitdata/nctest', 'rb') as f:
    NCtest=pickle.load(f) 

In [24]:
def load_logger(logdir):
    logger = logging.getLogger(__name__)
    logger.setLevel(logging.DEBUG)
    format = logging.Formatter("%(asctime)s - %(name)s - %(levelname)s - %(message)s")
    
    # Print to Stdout
    ch = logging.StreamHandler(sys.stdout)
    ch.setFormatter(format)
    logger.addHandler(ch)

    # Print to Log file
    fh = logging.FileHandler(os.path.join(logdir, 'log_' + str(uuid.uuid4())))
    fh.setFormatter(format)
    logger.addHandler(fh)

    return logger

def format_to_deepsurv(x, y):
    return {
        'x': x,
        'e': y[:,0].astype(np.int32),
        't': y[:,1].astype(np.float32)
    }

In [25]:
def load_box_constraints(file):
    with open(file, 'rb') as fp:
        return json.loads(fp.read())

In [44]:
def save_call_log(file, call_log):
    with open(file, 'wb') as fp:
        pickle.dump(call_log, fp)

def get_objective_function(num_epochs, logdir, update_fn = lasagne.updates.sgd):
    '''
    Returns the function for Optunity to optimize. The function returned by get_objective_function
    takes the parameters: x_train, y_train, x_test, and y_test, and any additional kwargs to 
    use as hyper-parameters.

    The objective function runs a DeepSurv model on the training data and evaluates it against the
    a validation set. The result of the function call is the validation concordance index 
    (which Optunity tries to optimize)
    '''
    def format_to_deepsurv(x, y):
        return {
            'x': x,
            'e': y[:,0].astype(np.int32),
            't': y[:,1].astype(np.float32)
        }

    def get_hyperparams(x,params):
        hyperparams = {
            'batch_norm': False,
            'activation': 'rectify',
            'standardize': True
        }

        if 'num_layers' in params and 'num_nodes' in params:
            params['hidden_layers_sizes'] = [int(params['num_nodes'])] * int(params['num_layers'])
            del params['num_layers']
            del params['num_nodes']

        if 'learning_rate' in params:
            params['learning_rate'] = 10 ** params['learning_rate']
        
        if 'n_in' in params:
            params['n_in']= int(np.floor(params['n_in']))
        
            
            

        hyperparams.update(params)
        return hyperparams

    def train_deepsurv(x_train,y_train,x_test,y_test,
        **kwargs):
        hyperparams = get_hyperparams(x_train,kwargs)
        x_train=x_train[:,0:hyperparams['n_in']]
        x_test=x_test[:,0:hyperparams['n_in']]
     
        
        # Standardize the datasets
        train_mean = x_train.mean(axis = 0)
        train_std = x_train.std(axis = 0)

        x_train = (x_train - train_mean) / train_std
        x_test = (x_test - train_mean) / train_std

        train_data = format_to_deepsurv(x_train, y_train)
        valid_data = format_to_deepsurv(x_test, y_test)

        

        # Set up Tensorboard loggers
        model_id = str(hash(str(hyperparams)))
        run_id = model_id + '_' + str(uuid.uuid4())
        logger = TensorboardLogger('hyperparam_search', 
            os.path.join(logdir,"tensor_logs", model_id, run_id))

        network = deep_surv.DeepSurv(**hyperparams)
        metrics = network.train(train_data, n_epochs = num_epochs, logger=logger, 
            update_fn = update_fn, verbose = False)

        result = network.get_concordance_index(**valid_data)
        main_logger.info('Run id: %s | %s | C-Index: %f | Train Loss %f' % (run_id, str(hyperparams), result, metrics['loss'][-1][1]))
        return result

    return train_deepsurv


## Find the optimal hyper-parameters using training data and save them in opt_params


In [45]:
NUM_EPOCHS = 100
NUM_FOLDS = 8
logdir='/home/arash/ProjectVR/logs'


In [46]:
hyperparams = {
    'batch_norm': False,
    'activation': 'rectify',
    'standardize': True
}

In [47]:
if 'learning_rate' in hyperparams:
    hyperparams['learning_rate'] = 10 ** hyperparams['learning_rate']
 

In [48]:
main_logger = load_logger(logdir)


#    main_logger.debug('Loading dataset: ' + args.dataset)
box_constraints = load_box_constraints('/home/arash/ProjectVR/box_constraints.0.json')

In [49]:
opt_fxn = get_objective_function(NUM_EPOCHS, logdir, 
                                 utils.get_optimizer_from_str('adam'))

opt_fxn = optunity.cross_validated(x=xtrain, y=ytrain, num_folds=NUM_FOLDS)(opt_fxn)


opt_params, call_log, _ = optunity.maximize(opt_fxn, num_evals=20,
        solver_name='sobol',
        **box_constraints)

WARNING (theano.tensor.blas): We did not find a dynamic library in the library_dir of the library we use for blas. If you use ATLAS, make sure to compile it with dynamics library.
WARNING (theano.tensor.blas): We did not find a dynamic library in the library_dir of the library we use for blas. If you use ATLAS, make sure to compile it with dynamics library.


2019-10-30 20:10:55,113 - __main__ - INFO - Run id: -1883527428915230341_4a1969c2-15b1-4930-ac5d-14deaa6dc2c9 | {'batch_norm': False, 'activation': 'rectify', 'standardize': True, 'lr_decay': 7.654296875000002e-05, 'learning_rate': 0.00025114346102550136, 'momentum': 0.8000400390624999, 'L2_reg': 2.5728564453125, 'dropout': 0.3740234375, 'n_in': 26, 'hidden_layers_sizes': [87, 87, 87]} | C-Index: 0.553720 | Train Loss 10.412197
2019-10-30 20:10:55,113 - __main__ - INFO - Run id: -1883527428915230341_4a1969c2-15b1-4930-ac5d-14deaa6dc2c9 | {'batch_norm': False, 'activation': 'rectify', 'standardize': True, 'lr_decay': 7.654296875000002e-05, 'learning_rate': 0.00025114346102550136, 'momentum': 0.8000400390624999, 'L2_reg': 2.5728564453125, 'dropout': 0.3740234375, 'n_in': 26, 'hidden_layers_sizes': [87, 87, 87]} | C-Index: 0.553720 | Train Loss 10.412197
2019-10-30 20:10:55,113 - __main__ - INFO - Run id: -1883527428915230341_4a1969c2-15b1-4930-ac5d-14deaa6dc2c9 | {'batch_norm': False, 'a

2019-10-30 20:11:01,571 - __main__ - INFO - Run id: 6497059791788648146_55283f8d-1972-4b90-8214-d760130e67c3 | {'batch_norm': False, 'activation': 'rectify', 'standardize': True, 'lr_decay': 0.00060248046875, 'learning_rate': 0.0001888738682406685, 'momentum': 0.7458994140624999, 'L2_reg': 1.5008720703124998, 'dropout': 0.2131484375, 'n_in': 21, 'hidden_layers_sizes': [29]} | C-Index: 0.529301 | Train Loss 8.971298
2019-10-30 20:11:02,759 - __main__ - INFO - Run id: 6497059791788648146_246cfc4a-b89f-40bd-8aaa-c12d35b85c7b | {'batch_norm': False, 'activation': 'rectify', 'standardize': True, 'lr_decay': 0.00060248046875, 'learning_rate': 0.0001888738682406685, 'momentum': 0.7458994140624999, 'L2_reg': 1.5008720703124998, 'dropout': 0.2131484375, 'n_in': 21, 'hidden_layers_sizes': [29]} | C-Index: 0.505681 | Train Loss 8.777582
2019-10-30 20:11:02,759 - __main__ - INFO - Run id: 6497059791788648146_246cfc4a-b89f-40bd-8aaa-c12d35b85c7b | {'batch_norm': False, 'activation': 'rectify', 'sta

2019-10-30 20:11:08,255 - __main__ - INFO - Run id: -8142789334062661712_4273f174-ff7a-49c7-951d-8f034876c8f8 | {'batch_norm': False, 'activation': 'rectify', 'standardize': True, 'lr_decay': 0.00035498046875, 'learning_rate': 1.932733057956347e-05, 'momentum': 0.8325244140624999, 'L2_reg': 0.2757470703124999, 'dropout': 0.1141484375, 'n_in': 15, 'hidden_layers_sizes': [96]} | C-Index: 0.502393 | Train Loss 6.372370
2019-10-30 20:11:10,144 - __main__ - INFO - Run id: -4909736832700930267_6b0812ce-58da-44a6-bb64-42cbe62e941a | {'batch_norm': False, 'activation': 'rectify', 'standardize': True, 'lr_decay': 0.00047873046875, 'learning_rate': 6.32662182206732e-07, 'momentum': 0.6159619140624999, 'L2_reg': 0.8883095703124999, 'dropout': 0.0646484375, 'n_in': 18, 'hidden_layers_sizes': [18, 18, 18]} | C-Index: 0.509166 | Train Loss 7.273788
2019-10-30 20:11:10,144 - __main__ - INFO - Run id: -4909736832700930267_6b0812ce-58da-44a6-bb64-42cbe62e941a | {'batch_norm': False, 'activation': 'rect

2019-10-30 20:11:16,454 - __main__ - INFO - Run id: 8754511770755017632_d8695b41-e558-44ed-a306-3a365b057ccf | {'batch_norm': False, 'activation': 'rectify', 'standardize': True, 'lr_decay': 0.00072623046875, 'learning_rate': 6.182610327435458e-06, 'momentum': 0.8758369140625, 'L2_reg': 2.1134345703124997, 'dropout': 0.36164843750000003, 'n_in': 12, 'hidden_layers_sizes': [85, 85, 85]} | C-Index: 0.484901 | Train Loss 10.188579
2019-10-30 20:11:18,091 - __main__ - INFO - Run id: 8754511770755017632_0862feff-4cb3-497d-a805-363319796a06 | {'batch_norm': False, 'activation': 'rectify', 'standardize': True, 'lr_decay': 0.00072623046875, 'learning_rate': 6.182610327435458e-06, 'momentum': 0.8758369140625, 'L2_reg': 2.1134345703124997, 'dropout': 0.36164843750000003, 'n_in': 12, 'hidden_layers_sizes': [85, 85, 85]} | C-Index: 0.495613 | Train Loss 9.271617
2019-10-30 20:11:18,091 - __main__ - INFO - Run id: 8754511770755017632_0862feff-4cb3-497d-a805-363319796a06 | {'batch_norm': False, 'act

2019-10-30 20:11:23,427 - __main__ - INFO - Run id: -8039894234906891778_c209ce51-775d-4b2a-91d6-08d845df8d65 | {'batch_norm': False, 'activation': 'rectify', 'standardize': True, 'lr_decay': 0.00016935546875000002, 'learning_rate': 1.1446477923516102e-07, 'momentum': 0.7675556640624999, 'L2_reg': 4.8699658203125, 'dropout': 0.0398984375, 'n_in': 11, 'hidden_layers_sizes': [90, 90]} | C-Index: 0.494710 | Train Loss 14.262037
2019-10-30 20:11:25,213 - __main__ - INFO - Run id: -8477846008211671645_2ed6a931-e5dc-46fc-850d-6d549f5496da | {'batch_norm': False, 'activation': 'rectify', 'standardize': True, 'lr_decay': 0.00066435546875, 'learning_rate': 1.093130155301593e-05, 'momentum': 0.9408056640625, 'L2_reg': 2.4197158203124998, 'dropout': 0.2378984375, 'n_in': 22, 'hidden_layers_sizes': [46, 46, 46]} | C-Index: 0.516433 | Train Loss 10.330717
2019-10-30 20:11:25,213 - __main__ - INFO - Run id: -8477846008211671645_2ed6a931-e5dc-46fc-850d-6d549f5496da | {'batch_norm': False, 'activation

2019-10-30 20:11:30,951 - __main__ - INFO - Run id: 6562146052299980092_ba4e6b2e-4e03-4a1a-81b7-855b1f229e54 | {'batch_norm': False, 'activation': 'rectify', 'standardize': True, 'lr_decay': 0.00041685546875, 'learning_rate': 0.00010682474751099208, 'momentum': 0.6809306640624999, 'L2_reg': 1.1945908203125, 'dropout': 0.1388984375, 'n_in': 28, 'hidden_layers_sizes': [68, 68]} | C-Index: 0.496310 | Train Loss 8.422617
2019-10-30 20:11:32,323 - __main__ - INFO - Run id: 6562146052299980092_65f28e73-b763-4cb3-b030-71673afbe499 | {'batch_norm': False, 'activation': 'rectify', 'standardize': True, 'lr_decay': 0.00041685546875, 'learning_rate': 0.00010682474751099208, 'momentum': 0.6809306640624999, 'L2_reg': 1.1945908203125, 'dropout': 0.1388984375, 'n_in': 28, 'hidden_layers_sizes': [68, 68]} | C-Index: 0.499534 | Train Loss 8.358720
2019-10-30 20:11:32,323 - __main__ - INFO - Run id: 6562146052299980092_65f28e73-b763-4cb3-b030-71673afbe499 | {'batch_norm': False, 'activation': 'rectify', 

2019-10-30 20:11:38,128 - __main__ - INFO - Run id: -8320422632531658821_70704ec6-33c9-4d8d-a5d5-bed55f75eb30 | {'batch_norm': False, 'activation': 'rectify', 'standardize': True, 'lr_decay': 0.00078810546875, 'learning_rate': 0.0003339426390923385, 'momentum': 0.6376181640624999, 'L2_reg': 3.0322783203125, 'dropout': 0.2873984375, 'n_in': 14, 'hidden_layers_sizes': [79, 79, 79]} | C-Index: 0.539837 | Train Loss 12.162933
2019-10-30 20:11:39,338 - __main__ - INFO - Run id: -6857521105471259914_79354734-3510-4576-bc3b-1552bffb99b4 | {'batch_norm': False, 'activation': 'rectify', 'standardize': True, 'lr_decay': 0.00054060546875, 'learning_rate': 3.5782598462943475e-07, 'momentum': 0.7242431640624999, 'L2_reg': 1.8071533203124999, 'dropout': 0.3863984375, 'n_in': 31, 'hidden_layers_sizes': [57]} | C-Index: 0.503712 | Train Loss 8.258920
2019-10-30 20:11:39,338 - __main__ - INFO - Run id: -6857521105471259914_79354734-3510-4576-bc3b-1552bffb99b4 | {'batch_norm': False, 'activation': 'rect

2019-10-30 20:11:44,024 - __main__ - INFO - Run id: 1653309244707763173_fc9f8dc8-bd46-4234-8395-786ca33bc28e | {'batch_norm': False, 'activation': 'rectify', 'standardize': True, 'lr_decay': 6.107421875000001e-05, 'learning_rate': 5.361630425778916e-06, 'momentum': 0.6322041015625, 'L2_reg': 1.1180205078124998, 'dropout': 0.30596093750000003, 'n_in': 22, 'hidden_layers_sizes': [94]} | C-Index: 0.471298 | Train Loss 8.840106
2019-10-30 20:11:45,206 - __main__ - INFO - Run id: 1653309244707763173_7497f0d7-1ad5-4452-94be-7918cf85473f | {'batch_norm': False, 'activation': 'rectify', 'standardize': True, 'lr_decay': 6.107421875000001e-05, 'learning_rate': 5.361630425778916e-06, 'momentum': 0.6322041015625, 'L2_reg': 1.1180205078124998, 'dropout': 0.30596093750000003, 'n_in': 22, 'hidden_layers_sizes': [94]} | C-Index: 0.530744 | Train Loss 8.475849
2019-10-30 20:11:45,206 - __main__ - INFO - Run id: 1653309244707763173_7497f0d7-1ad5-4452-94be-7918cf85473f | {'batch_norm': False, 'activation

2019-10-30 20:11:51,566 - __main__ - INFO - Run id: -5028161358728923789_c16e1da0-4e80-4fe3-873d-a6cef7edc455 | {'batch_norm': False, 'activation': 'rectify', 'standardize': True, 'lr_decay': 0.00080357421875, 'learning_rate': 5.486518841898849e-07, 'momentum': 0.8920791015624999, 'L2_reg': 2.3431455078125, 'dropout': 0.008960937500000002, 'n_in': 28, 'hidden_layers_sizes': [28, 28]} | C-Index: 0.515721 | Train Loss 9.851717


In [50]:
opt_params

{'lr_decay': 0.00055607421875,
 'learning_rate': -3.290703125,
 'num_nodes': 50.3013671875,
 'num_layers': 3.02787109375,
 'momentum': 0.8054541015624999,
 'L2_reg': 3.5682705078125,
 'dropout': 0.1079609375,
 'n_in': 11.0484697265625}

## train a network based on opt_params on training data and save the model and weights (previous sections used cross-validation to find hyper-parameters, weights could not be saved)

In [ ]:
NUM_EPOCHS=4000
hyperparams = {
    'batch_norm': True,
    'activation': 'rectify',
    'standardize': False,
    'n_in':xtrain.shape[1]
}

hyperparams.update(opt_params)

if 'num_layers' in hyperparams and 'num_nodes' in hyperparams:
    hyperparams['hidden_layers_sizes'] = [int(hyperparams['num_nodes'])] * int(hyperparams['num_layers'])
    del hyperparams['num_layers']
    del hyperparams['num_nodes']

if 'learning_rate' in hyperparams:
    hyperparams['learning_rate'] = 10 ** hyperparams['learning_rate']
 

In [ ]:
### Standardizing
#trainsub_mean = xtrainsub.mean(axis = 0)
#trainsub_std = xtrainsub.std(axis = 0)
#xtrainsub = (xtrainsub - trainsub_mean) / trainsub_std

trainsub_data=format_to_deepsurv(xtrainsub, ytrainsub)

#valsub_mean = xvalsub.mean(axis = 0)
#valsub_std = xvalsub.std(axis = 0)
#xvalsub = (xvalsub - valsub_mean) / valsub_std

valsub_data=format_to_deepsurv(xvalsub, yvalsub)


network = deep_surv.DeepSurv(**hyperparams)

In [ ]:
metrics = network.train(trainsub_data,valsub_data, n_epochs = NUM_EPOCHS, update_fn = lasagne.updates.sgd, verbose = True)

In [ ]:
network.get_concordance_index(xvalsub,yvalsub[:,1],yvalsub[:,0])

## apply the trained network on test set and check the feature importance for generalization using  [permutation importance](https://eli5.readthedocs.io/en/latest/blackbox/permutation_importance.html)


In [ ]:
colnames=test.iloc[:,1:-1].columns
def score(X, y):
    X=pd.DataFrame(X)
    X.columns=colnames
    data=X
    data['Wait Time (s)']=y.values
    data['E']=1
    test = dataframe_to_deepsurv_ds(data, event_col = 'E', time_col= 'Wait Time (s)')
    xtest=test['x']
    xtest = (xtest - trainsub_mean) / trainsub_std              
    etest=test['e']
    ttest=test['t']
    ytest = np.column_stack((etest, ttest))
    valid_data = format_to_deepsurv(xtest, ytest)               #fix: network is trained again on test data
    ci=network.get_concordance_index(**valid_data)
    return ci


st=time.time()

base_score, score_decreases = get_score_importances(score, test.iloc[:,1:-1].values, test.iloc[:,0])
                                                    #,columns_to_shuffle=range(0,2))
f=time.time()

feature_importances_mean = np.mean(score_decreases, axis=0)
feature_importances_std = np.std(score_decreases, axis=0)

perimportance=pd.DataFrame(data=feature_importances_mean,index=colnames,columns=['mean'])
perimportance['std']=feature_importances_std



perimportance=perimportance.sort_values(by=['mean'])



def plot_feature_importances(imp):
    plt.figure(figsize=(16,12))
    n_features = imp.shape[0]
    plt.barh(range(n_features), perimportance.iloc[:,0].values, align='center')
    plt.yticks(np.arange(n_features), colnames)
    plt.xlabel("Feature importance")
    plt.ylabel("Feature")
    plt.ylim(-1, n_features)

plot_feature_importances(perimportance)
#plt.savefig('Decision Tree feature_importance')

In [ ]:
NCtrain.iloc[:,1:-1].columns

## Using Shap for feature importance

In [ ]:
#using train set for interpretability
dfxtrainsub=pd.DataFrame(xtrain)
dfxtrainsub.columns=NCtrain.iloc[:,1:-1].columns

#using zeros as background data
backgrounddata=pd.DataFrame(np.zeros(32)).T         
backgrounddata.columns=NCtrain.iloc[:,1:-1].columns

explainer = shap.KernelExplainer(network.predict_risk,backgrounddata)
shap_values = explainer.shap_values(dfxtrainsub)

In [ ]:
rework=pd.DataFrame(shap_values[0])
rework.columns=features

In [ ]:
reworkclean=rework.drop(columns=['One Way Road', 'Two way with Median Road', 'Night Time', 'Age between 18 - 24',
    'Employed Person',
       'Student', 'Owns a Driving License', 'Main Mode: Car',
      'Main Mode: Walking', 'Sometimes Walks to Work',
       'Always Walks to Work', 'Sometimes Walks for shopping',
       'Speed Limit is Low', 'Speed Limit is high',
       'Minimum Gap is Low',
       'Mean Arrival Rate is Low', 'Does not own a car',
       'Has more than one car'])

In [ ]:
reworkclean.columns

In [ ]:
len(reworkclean.values)

In [ ]:
features=['Snowy Weather', 'Age between 25 - 29', 'Age between 30 - 39',
       'Gender: Female', 'Main Mode: Public Transit',
       'Always Walks for shopping', 'Has VR experinece', 'Lane Width is short',
       'Lane Width is long', 'Minimum Gap is High',
       'Mean Arrival Rate is High', 'Fully Automated Condition',
       'Mixed Automated Condition', 'Age is over 50']

In [ ]:
xx=shap.summary_plot(reworkclean.values, features,max_display=32)

In [ ]:
shap.summary_plot(reworkclean.values, features,max_display=32,show=False)

In [ ]:
import matplotlib.pyplot as pl
shap.summary_plot(reworkclean.values, features,max_display=32,show=False)
pl.savefig('shap.png',bbox_inches='tight')


In [ ]:
shap_df=pd.DataFrame(shap_values[0])
shap_df.columns=NCtrain.iloc[:,1:-1].columns

mean=shap_df[shap_df!=0].mean()              #mean of shap values for each feature (non-zeros only

nonzeros=shap_df.astype(bool).sum(axis=0)        #count of non-zeros shap values for each feature

std=shap_df[shap_df!=0].std()                #standard deviation of shap values for each feature

featuresshap=pd.DataFrame()

featuresshap['count1']=nonzeros
featuresshap['bg0mean']=mean                 #bg0: zeros set as background data

featuresshap['bg0std']=std

featuresshap['bg0absmean']=abs(mean)

featuresshap.sort_values(by=['bg0absmean'])

In [ ]:
#using ones as background data
backgrounddata2=pd.DataFrame(np.ones(42)).T         
backgrounddata2.columns=NCtrain.iloc[:,1:-1].columns

explainer2 = shap.KernelExplainer(network.predict_risk,backgrounddata2)
shap_values2 = explainer2.shap_values(dfxtrainsub)





In [ ]:
shap.summary_plot(shap_values2[0], dfxtrainsub,max_display=42)

In [ ]:
shap_values2=pd.DataFrame(shap_values2[0])
shap_values2.columns=NCtrain.iloc[:,1:-1].columns

mean2=shap_values2[shap_values2!=0].mean()              #mean of shap values for each feature (non-zeros only

std2=shap_values2[shap_values2!=0].std()                #standard deviation of shap values for each feature

featuresshap['bg1mean']=mean2                           #bg1: ones set as background data

featuresshap['bg1std']=std2

featuresshap['bg1absmean']=abs(mean2)

In [ ]:
featuresshap











